## Import Module

In [0]:
import tensorflow as tf
import numpy as np
import scipy.special
import matplotlib.pyplot as plt
%matplotlib inline

## Dataset

In [0]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

### Analyze Dataset

In [0]:
image = np.asfarray(x_train[0]).reshape((28, 28))
plt.imshow(image, cmap='gray', interpolation='None')

## Model

In [0]:
class NeuralNetwork:
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        self.inodes = input_nodes
        self.hnodes = hidden_nodes
        self.onodes = output_nodes

        self.wih = np.random.normal(0.0, pow(self.hnodes, -0.5), (self.hnodes, self.inodes))
        self.who = np.random.normal(0.0, pow(self.onodes, -0.5), (self.onodes, self.hnodes))

        self.lr = learning_rate

        self.activation_function = lambda x: scipy.special.expit(x)
    
    def train(self, input_list, target_list):
        inputs = np.array(input_list, ndmin=2).T
        targets = np.array(target_list, ndmin=2).T

        hidden_inputs = np.dot(self.wih, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)

        final_inputs = np.dot(self.who, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)

        output_errors = targets - final_outputs
        hidden_errors = np.dot(self.who.T, output_errors)

        self.who += self.lr * np.dot((output_errors * final_outputs * (1.0 - final_outputs)), np.transpose(hidden_outputs))
        self.wih += self.lr * np.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), np.transpose(inputs))
    
    def query(self, input_list):
        inputs = np.array(input_list, ndmin=2).T

        hidden_inputs = np.dot(self.wih, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        
        final_inputs = np.dot(self.who, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)

        return final_outputs

## Set Initial Parameter

In [0]:
input_nodes = 784
hidden_nodes = 200
output_nodes = 10

learning_rate = 0.1

epochs = 5

## Training

In [0]:
nn = NeuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

for e in range(epochs):
    for i in range(len(x_train)):
        input = np.asfarray(x_train[i]).flatten() / 255.0 * 0.99 + 0.01
        target = np.zeros(output_nodes) + 0.01
        target[int(y_train[i])] = 0.99
        nn.train(input, target)

## Test

In [0]:
score_list = []
for i in range(len(x_test)):
    expected = int(y_test[i])
    input = np.asfarray(x_test[i]).flatten() / 255.0 * 0.99 + 0.01
    output = nn.query(input)
    predicted = np.argmax(output)
    if predicted == expected:
        score_list.append(1)
    else:
        score_list.append(0)
score_array = np.asarray(score_list)
print(f"Performance = {score_array.sum() / score_array.size}")